# TITLE: CLUSTERING NEIGHBORHOODS IN TORONTO BASED ON FOURSQUARE VENUES

This is part of a Coursera machine learning course. This exercise have 3 parts:


- In Part 1 we scrape a table from wikipedia that contains the postal codes of Toronto neighborhoods and create a Pandas Data Frame

- In Part 2 we add to the dataframe the coordinates (latitude,longitude) of every postal code

- In Part 3 we select the closest neighborhoods to the lake, and then using the Foursquare api we get the top 100 venues of each neighborhood. With this information, using sklearn k-means algorithm, the neighborhoods are grouped into 3 clusters.


# PART 1: CREATING DATAFRAME FROM WIKIPEDIA

In [1]:
#importing libraries

import pandas as pd
import requests
from pandas.io.html import read_html

In [2]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#scrape table from wikipedia

df = read_html(page,  attrs={"class":"wikitable"})[0]
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
#Drop rows with Not assigned Borough

df_na = df.drop(df[df['Borough'] == 'Not assigned'].index)
df_na.reset_index(drop=True,inplace=True)

df_na.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
#check out if there are null values in columns

print('Neighborhood null values?:', df_na['Neighborhood'].isnull().any())
print('Postal code null values?:',df_na['Postal code'].isnull().any())
print('Borough null values?:',df_na['Borough'].isnull().any())

Neighborhood null values?: False
Postal code null values?: False
Borough null values?: False


In [6]:
#check out if there are 'Not assigned' in Neighborhood

df_na[df_na['Neighborhood']=='Not assigned'].any()

Postal code     False
Borough         False
Neighborhood    False
dtype: bool

In [7]:
#check out if there are duplicated values in Postal code

df_na.duplicated(subset='Postal code').any()

False

In [8]:
#checks out if there are duplicated values in Neighborhood

mask = df_na.duplicated(subset='Neighborhood',keep=False).tolist()


In [9]:
#there are Neighborhoods with the same name (but different Postal code)

df_duplicated = df_na[mask]
df_duplicated

,Postal code,Borough,Neighborhood
7,M3B,North York,Don Mills
13,M3C,North York,Don Mills
40,M3K,North York,Downsview
46,M3L,North York,Downsview
53,M3M,North York,Downsview
59,M2N,North York,Willowdale
60,M3N,North York,Downsview
72,M2R,North York,Willowdale


In [10]:
#Lets add the postal code to these neighborhood names in DataFrame to differentiate them, as I do not know their specific name

for index,row in df_duplicated.iterrows():
    
    df_na.loc[index,'Neighborhood'] = df_na.loc[index,'Neighborhood'] + " " + df_na.loc[index,'Postal code']



In [11]:
#final df (see row 7: Don Mills M3B)

print(df_na.shape)

df_na.head(10)

(103, 3)


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills M3B
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
#check integrity of df

df_na.groupby('Borough').count()

,Postal code,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Scarborough,17,17
West Toronto,6,6


In [13]:
#check integrity of df

df_na.describe(include='all')

,Postal code,Borough,Neighborhood
count,103,103,103
unique,103,10,103
top,M3C,North York,West Deane Park / Princess Gardens / Martin Gr...
freq,1,24,1


# PART 2: ADD THE LATITUDE AND LONGITUDE COORDINATES OF EACH NEIGHBORHOOD 

In [14]:
#Download the geospatial_data csv

!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [15]:
#Read it to a dataframe

geo_data = pd.read_csv('geospatial_data.csv')

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
geo_data.shape

(103, 3)

In [17]:
#check out if there are null values in Postal Code

geo_data['Postal Code'].isnull().any()

False

In [18]:
#check out if there are duplicated values in Postal Code

geo_data.duplicated(subset='Postal Code').any()

False

In [19]:
#rename 'Postal code' column in geo_data to match the same column name than df_na

geo_data.rename(columns = {'Postal Code':'Postal code'}, inplace = True)

geo_data.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
#Compare two columns of postal code in both dataframes

df_na['Postal code'].isin(geo_data['Postal code']).value_counts()

True    103
Name: Postal code, dtype: int64

In [21]:
#Merge both dataframes

Toronto_df = pd.merge(df_na, geo_data, on=['Postal code'],how='inner')

In [22]:
Toronto_df.head(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills M3B,43.745906,-79.352188
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# PART 3: CLUSTERING NEIGHBORHOODS IN TORONTO

In [23]:
#import libraries

import json # library to handle JSON files
import numpy as np
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



In [24]:
#get the coordinates from Toronto area

latitude = Toronto_df['Latitude'][0]
longitude = Toronto_df['Longitude'][0]


# create map of Toronto using latitude and longitude values and zoom 10
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
#This is the example proposed for coursera. We are going to code another example instead.

#Toronto = Toronto_df[Toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True) 


### We are going to select the closest neighborhoods to the lake and then compare them

In [26]:
#Lets create a new dataframe with the neighborhoods close to the lake. 

#we plot a line from Mimico NW neighborhood to Rouge Hill. First we get the coordinates from both neighborhoods:

Mimico_coord = Toronto_df[Toronto_df['Neighborhood'].str.contains('Mimico NW')].loc[:,'Latitude':'Longitude'].values.tolist()[0]

Rouge_coord = Toronto_df[Toronto_df['Neighborhood'].str.contains('Rouge Hill')].loc[:,'Latitude':'Longitude'].values.tolist()[0]

#Now we update the map with the line

points=[tuple(Mimico_coord),tuple(Rouge_coord)]
names=['Mimico NW','Rouge Hill']


for each,label in zip (points,names):
    folium.Marker(each, popup=label).add_to(map_toronto)
 
folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(map_toronto)


map_toronto


In [27]:
#We want to compare the neighborhoods below this red line, which are the ones close to the lake

#First we get the equation of the line that we plotted above


x=[Mimico_coord[1],Rouge_coord[1]]
y=[Mimico_coord[0],Rouge_coord[0]]

coefficients = np.polyfit(x, y, 1)
p = np.poly1d(coefficients)
p

poly1d([ 0.43188157, 77.97249502])

In [28]:
#We make a copy of initial DF and add a column with the function of the line for each neighborhood (Latitude_calc)

Toronto_calc = Toronto_df.copy()

Toronto_calc['Latitude_calc'] = p(Toronto_calc['Longitude'])

Toronto_calc.head(5)

,Postal code,Borough,Neighborhood,Latitude,Longitude,Latitude_calc
0,M3A,North York,Parkwoods,43.753259,-79.329656,43.711478
1,M4A,North York,Victoria Village,43.725882,-79.315572,43.717561
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,43.698099
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,43.653128
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,43.685636


In [29]:
#We add a new column with the difference between latitude calculated and real. Positive values are neighborhoods above the line

Toronto_calc['Result']=Toronto_calc['Latitude_calc']-Toronto_calc['Latitude']

In [30]:
#Create the final dataframe: Toronto_lake

Toronto_lake = Toronto_calc[Toronto_calc['Result']>-0.00001]

Toronto_lake.drop(['Latitude_calc','Result'],axis=1,inplace=True) 

Toronto_lake.reset_index(drop=True,inplace=True)

print('There are', Toronto_lake.shape[0], 'neighborhoods in the new dataframe')

Toronto_lake.head(5)

There are 42 neighborhoods in the new dataframe


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
4,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497


In [31]:
#Now we create a new map only with the neighborhoods close to the lake

latitude = Toronto_lake['Latitude'][10]
longitude = Toronto_lake['Longitude'][10]


map_lake = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_lake['Latitude'], Toronto_lake['Longitude'], Toronto_lake['Borough'], Toronto_lake['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lake)  
    
    
    
points=[tuple(Mimico_coord),tuple(Rouge_coord)]
names=['Mimico NW','Rouge Hill']


for each,label in zip (points,names):
    folium.Marker(each, popup=label).add_to(map_lake)
 
folium.PolyLine(points, color="red", weight=2.5, opacity=1).add_to(map_lake)



map_lake

## FOURSQUARE API

In [32]:
# The code was removed by Watson Studio for sharing.

In [33]:
# This is a function to get the top venues that are in every neighborhood

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
# Let's call the function to get the top 100 venues of neighborhoods in Toronto_lake

LIMIT=100

lake_venues = getNearbyVenues(names=Toronto_lake['Neighborhood'],
                                   latitudes=Toronto_lake['Latitude'],
                                   longitudes=Toronto_lake['Longitude']
                                  )


Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Rouge Hill / Port Union / Highland Creek
Woodbine Heights
St. James Town
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Scarborough Village
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
Golden Mile / Clairlea / Oakridge
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Cliffside / Cliffcrest / Scarborough Village West
Studio District
Birch Cliff / Cliffside West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst 

In [35]:
print('There are', lake_venues.shape[0], 'venues')
lake_venues.head()

There are 1526 venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [36]:
# Now we check how many venues do we have in each neighborhood

lake_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alderwood / Long Branch,8,8,8,8,8,8
Berczy Park,55,55,55,55,55,55
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,24,24,24,24,24,24
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77


In [37]:
#check how many categories of venues are

print('There are {} uniques categories.'.format(len(lake_venues['Venue Category'].unique())))


There are 233 uniques categories.


In [38]:
#There are an average of 36 venues per neighborhood, and neighborhoods with only 1 venue

lake_venues.groupby('Neighborhood')['Venue'].count().describe()

count     42.000000
mean      36.333333
std       34.483412
min        1.000000
25%        8.250000
50%       22.000000
75%       55.000000
max      100.000000
Name: Venue, dtype: float64

In [39]:
# We are going to drop neighborhoods with less than 4 venues

df_count = lake_venues.groupby('Neighborhood').count()

df_count = df_count[df_count['Venue']<4].index.tolist()

print('We are going to drop', len(df_count),'neighborhoods')

print('Name of neighborhoods with less than 4 venues:', df_count)


for value in df_count:
    
   lake_venues.drop(lake_venues[lake_venues['Neighborhood']==value].index,inplace=True)


lake_venues.reset_index(drop=True,inplace=True)

print(lake_venues.shape)

lake_venues.groupby('Neighborhood').count()

We are going to drop 5 neighborhoods
Name of neighborhoods with less than 4 venues: ['Cliffside / Cliffcrest / Scarborough Village West', 'East Toronto', "Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East", 'Rouge Hill / Port Union / Highland Creek', 'Scarborough Village']
(1517, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alderwood / Long Branch,8,8,8,8,8,8
Berczy Park,55,55,55,55,55,55
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,24,24,24,24,24,24
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77


In [40]:
# Create a new dataframe encoding venue cateories


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# one hot encoding
lake_onehot = pd.get_dummies(lake_venues[['Venue Category']], prefix="", prefix_sep="")

# rename Neighborhood category from foursquare

lake_onehot.rename({'Neighborhood':'Neighborhood FQ'},axis=1,inplace=True)

# add neighborhood column back to dataframe
lake_onehot['Neighborhood'] = lake_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lake_onehot.columns[-1]] + list(lake_onehot.columns[:-1])
lake_onehot = lake_onehot[fixed_columns]

lake_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood FQ,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [41]:
# confirm size of df

print(lake_onehot.shape)

(1517, 232)


In [42]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


lake_grouped = lake_onehot.groupby('Neighborhood').mean().reset_index()
lake_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood FQ,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Alderwood / Long Branch,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [43]:
# confirm size of df

print('There are', lake_grouped.shape[0], 'neigborhoods and', lake_grouped.shape[1]-1,'venue categories')

There are 37 neigborhoods and 231 venue categories


In [44]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in lake_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lake_grouped[lake_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood / Long Branch----
          venue  freq
0   Pizza Place  0.25
1           Gym  0.12
2  Skating Rink  0.12
3   Coffee Shop  0.12
4           Pub  0.12


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2            Beer Bar  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Birch Cliff / Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4          Movie Theater  0.00


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0       Nightclub  0.12
1            Café  0.12
2     Coffee Shop  0.08
3  Breakfast Spot  0.08
4             Gym  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0                  Park  0.06
1         Auto Workshop  0.06
2               Brewery  0.06
3  Fast Food Restaurant  0.06
4            Smoke Sho

In [45]:
#let's write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
#let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lake_grouped['Neighborhood']

for ind in np.arange(lake_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lake_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alderwood / Long Branch,Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Sandwich Place,Coffee Shop,American Restaurant,Curling Ice,Eastern European Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Restaurant,Jazz Club,Farmers Market
2,Birch Cliff / Cliffside West,Café,College Stadium,Skating Rink,General Entertainment,Curling Ice,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
3,Brockton / Parkdale Village / Exhibition Place,Nightclub,Café,Coffee Shop,Breakfast Spot,Bakery,Convenience Store,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
4,Business reply mail Processing CentrE,Park,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Light Rail Station,Skate Park


## CLUSTERING NEIGHBORHOODS

In [47]:
# set number of clusters
kclusters = 3

lake_grouped_clustering = lake_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lake_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [49]:
lake_merged = Toronto_lake

# merge neighborhoods_venues_sorted with toronto_lake to add latitude/longitude for each neighborhood
lake_merged = lake_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

lake_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Ice Cream Shop,Beer Store
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop,General Entertainment
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,1,Pizza Place,Fast Food Restaurant,Intersection,Athletics & Sports,Gastropub,Bank,Pharmacy,Gym / Fitness Center,Dog Run,Doner Restaurant
3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
5,M4C,East York,Woodbine Heights,43.695344,-79.318389,1,Skating Rink,Pharmacy,Beer Store,Spa,Curling Ice,Asian Restaurant,Cosmetics Shop,Park,Video Store,Comic Shop


In [50]:
# Drop Postal Code column

lake_merged.drop(lake_merged.columns[[0]],axis=1,inplace=True)

lake_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Ice Cream Shop,Beer Store
1,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop,General Entertainment
2,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,1,Pizza Place,Fast Food Restaurant,Intersection,Athletics & Sports,Gastropub,Bank,Pharmacy,Gym / Fitness Center,Dog Run,Doner Restaurant
3,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
5,East York,Woodbine Heights,43.695344,-79.318389,1,Skating Rink,Pharmacy,Beer Store,Spa,Curling Ice,Asian Restaurant,Cosmetics Shop,Park,Video Store,Comic Shop


### Let's create the map with the clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lake_merged['Latitude'], lake_merged['Longitude'], lake_merged['Neighborhood'], lake_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 0: A neighborhood close to the beach, with trail routes for hiking, with places to drink a coffee or eat something during the day and a great night atmosphere

In [52]:
lake_merged.loc[lake_merged['Cluster Labels'] == 0, lake_merged.columns[[1] + list(range(5, lake_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,The Beaches,Trail,Coffee Shop,Pub,Health Food Store,Neighborhood FQ,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop


### Cluster 1: Most of the neighborhoods analyzed are similar: Many places to drink a coffee, many places to eat, and also with other leisure services (park, sports...) 

In [53]:
lake_merged.loc[lake_merged['Cluster Labels'] == 1, lake_merged.columns[[1] + list(range(5, lake_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regent Park / Harbourfront,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Ice Cream Shop,Beer Store
1,Queen's Park / Ontario Provincial Government,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Mexican Restaurant,Juice Bar,Japanese Restaurant,Italian Restaurant,Hobby Shop,General Entertainment
2,Parkview Hill / Woodbine Gardens,Pizza Place,Fast Food Restaurant,Intersection,Athletics & Sports,Gastropub,Bank,Pharmacy,Gym / Fitness Center,Dog Run,Doner Restaurant
3,"Garden District, Ryerson",Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
5,Woodbine Heights,Skating Rink,Pharmacy,Beer Store,Spa,Curling Ice,Asian Restaurant,Cosmetics Shop,Park,Video Store,Comic Shop
6,St. James Town,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Gym,Seafood Restaurant,Hotel,Italian Restaurant,Department Store
7,Guildwood / Morningside / West Hill,Rental Car Location,Breakfast Spot,Medical Center,Mexican Restaurant,Bank,Intersection,Electronics Store,Deli / Bodega,Donut Shop,Doner Restaurant
9,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Restaurant,Jazz Club,Farmers Market
10,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Bar,Thai Restaurant
11,Christie,Grocery Store,Café,Park,Baby Store,Diner,Italian Restaurant,Restaurant,Candy Store,Coffee Shop,Nightclub


### Cluster 2: A neighborhood with nature: parks, playgrounds and trails. Also some places to eat

In [54]:
lake_merged.loc[lake_merged['Cluster Labels'] == 2, lake_merged.columns[[1] + list(range(5, lake_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Rosedale,Park,Playground,Trail,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
